In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the large PNG image
image = cv2.imread('/content/143HD_003.png')

# Resize the image to 25% of its original size
resized_image = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)

# Save the resized image
cv2.imwrite('resized_image.png', resized_image)

# Verify the resize by displaying
from google.colab.patches import cv2_imshow
cv2_imshow(resized_image)

In [ ]:
def adaptive_brightness_contrast(image):
    """
    Adjust brightness and contrast adaptively using histogram equalization.
    """
    # Convert to YUV color space
    yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    # Equalize the histogram of the Y channel
    yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
    # Convert back to BGR color space
    return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)

def remove_white_space(image):
    """
    Remove white space from an image by cropping to the bounding box of non-white areas.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Invert the image so white becomes black and vice versa
    inverted = cv2.bitwise_not(gray)

    # Threshold to create a binary image
    _, thresh = cv2.threshold(inverted, 1, 255, cv2.THRESH_BINARY)

    # Find contours of the non-white areas
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Get bounding box for the non-white regions
        x, y, w, h = cv2.boundingRect(np.concatenate(contours))

        # Crop the image to the bounding box
        cropped_image = image[y:y+h, x:x+w]
        return cropped_image
    return image  # Return original if no white space is detected

def dynamic_canny(image):
    """
    Perform edge detection using dynamic thresholds based on the image's median intensity.
    """
    median_val = np.median(image)
    lower = int(max(0, 0.7 * median_val))
    upper = int(min(255, 1.3 * median_val))
    return cv2.Canny(image, lower, upper)

def process_subimage(subimage):
    """
    Process a subimage to detect ROIs, balancing edge detection and low-contrast thresholding.
    """
    # Step 1: Adjust brightness and contrast adaptively
    adjusted_image = adaptive_brightness_contrast(subimage)

    # Step 2: Remove white space
    cropped_image = remove_white_space(adjusted_image)

    # Step 3: Convert to grayscale
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # Step 4: Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Step 5: Use dynamic Canny edge detection to find edges
    edges = dynamic_canny(blurred)

    # Step 6: Slightly dilate edges to connect close features
    kernel = np.ones((3, 3), np.uint8)  # Smaller kernel for precision
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)

    # Step 7: Apply threshold to enhance low-contrast areas
    _, thresholded = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)

    # Step 8: Combine edges and thresholded areas, with priority on edges
    combined = cv2.bitwise_or(edges_dilated, cv2.bitwise_and(thresholded, edges))

    # Step 9: Find contours from the combined image
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    rois = []
    for contour in contours:
        # Get bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)
        x_midpoint = x + w / 2
        y_midpoint = y + h / 2

        # Ignore very small contours (noise)
        if w > 5 and h > 5:  # Filter small boxes
            roi = {
                "x_midpoint": x_midpoint,
                "y_midpoint": y_midpoint,
                "x_length": w,
                "y_length": h,
            }
            rois.append(roi)

            # Draw bounding box for visualization
            cv2.rectangle(cropped_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the cropped image with bounding boxes
    cv2_imshow(cropped_image)

    return rois, cropped_image


def divide_image_into_grid_with_overlap(image, grid_size, overlap=0.1):
    """
    Divide the image into a grid with overlapping regions between subimages.
    """
    image_height, image_width = image.shape[:2]
    rows, cols = grid_size
    sub_height = image_height // rows
    sub_width = image_width // cols
    overlap_h = int(overlap * sub_height)
    overlap_w = int(overlap * sub_width)

    subimages = []
    for row in range(rows):
        for col in range(cols):
            # Calculate subimage coordinates with overlap
            x_start = max(0, col * sub_width - overlap_w)
            y_start = max(0, row * sub_height - overlap_h)
            x_end = min(image_width, x_start + sub_width + 2 * overlap_w)
            y_end = min(image_height, y_start + sub_height + 2 * overlap_h)

            subimage = image[y_start:y_end, x_start:x_end]
            subimages.append((subimage, x_start, y_start))

    return subimages

def process_image_with_grid(resized_image, grid_size, overlap=0.1):
    """
    Divide an image into a grid with overlap and process each subimage for ROIs.
    """
    image_height, image_width = resized_image.shape[:2]
    subimages = divide_image_into_grid_with_overlap(resized_image, grid_size, overlap)
    all_rois = []

    for subimage, x_offset, y_offset in subimages:
        # Process the subimage
        rois, processed_subimage = process_subimage(subimage)

        # Normalize ROIs to the original image
        for roi in rois:
            adjusted_roi = {
                "x_midpoint_normalized": round((roi["x_midpoint"] + x_offset) / image_width, 6),
                "y_midpoint_normalized": round((roi["y_midpoint"] + y_offset) / image_height, 6),
                "x_length_normalized": round(roi["x_length"] / image_width, 6),
                "y_length_normalized": round(roi["y_length"] / image_height, 6),
            }
            all_rois.append(adjusted_roi)

    return all_rois

# Example usage
image_path = '/content/resized_image.png'  # Replace with the uploaded image path
resized_image = cv2.imread(image_path)

# Validate image loading
if resized_image is not None:
    image_height, image_width = resized_image.shape[:2]
    grid_size = (image_height // 200, image_width // 200)  # Example grid size, made flexible
    overlap = 0.1  # 10% overlap between subimages
    rois = process_image_with_grid(resized_image, grid_size, overlap)
    print("Detected ROIs:")
    for i, roi in enumerate(rois):
        print(f"ROI {i + 1}: {roi}")
else:
    print("Error: Image could not be loaded.")
